# Low z LLS (Lehner+13, Wotta+16)

In [41]:
# imports
from scipy.io import readsav

import pyigm
from pyigm.abssys.lls import LLSSystem
from pyigm.surveys.llssurvey import LLSSurvey
pyigm_path = pyigm.__path__[0]

## Lehner+13

### Metals

From
http://iopscience.iop.org/0004-637X/770/2/138/suppdata/apj472363t2_mrt.txt

### Metallicity

In [18]:
l13w16 = readsav(pyigm_path+'/data/LLS/Literature/wotta16_final.save')

### RA/DEC

In [22]:
l13_radec = {'J0943+0531': 'J094331.61+053131.4', 
             'J1419+4207': 'J141910.20+420746.9', 
             'J1435+3604': 'J143511.53+360437.2', 
             'PG1116+215': 'J111908.67+211918.0',
             'PG1522+101': 'J152424.52+095829.1', 
             'PG1338+416': 'J134100.78+412314.0', 
             'HE0439-5254': 'J044012.04-524817.65',        
             'PG1407+265': 'J140923.91+261821.1', 
             'PG1216+069': 'J121920.92+063838.5', 
             'PKS0405-123': 'J040748.44-121136.7',
             'J1619+3342': 'J161916.54+334238.4', 
             'PKS0637-752': 'J063546.51-751616.7', 
             'SBS1122+594': 'J112553.79+591021.5',
             'HE0153-4520': 'J015513.22-450611.6', 
             'PHL1377': 'J023507.34-040205.3', 
             'PKS0552-640': 'J055224.51-640210.8',
             'PG1630+377': 'J163201.11+373750.0', 
             'PKS1302-102': 'J130533.01-103319.4', 
             'PG1206+459': 'J120858.00+454035.4', 
             'PG1634+706': 'J163428.99+703132.4'}

### $z_{\rm em}$

In [26]:
l13_zem =  {'J0943+0531': 0.56449,
             'J1419+4207': 0.87350,
             'J1435+3604': 0.42860,
             'PG1116+215': 0.17650,
             'PG1522+101': 1.32825,
             'PG1338+416': 1.21420,
             'HE0439-5254': 1.05300,
             'PG1407+265': 0.94,
             'PG1216+069': 0.3313,
             'PKS0405-123': 0.5726,
             'J1619+3342': 0.47093,
             'PKS0637-752': 0.651,
             'SBS1122+594': 0.85242, 
             'HE0153-4520': 0.451,
             'PHL1377': 1.4398,
             'PKS0552-640': 0.680,
             'PG1630+377': 1.47465,
             'PKS1302-102': 0.2784,
             'PG1206+459': 1.16200,
             'PG1634+706': 1.337 }

In [35]:
l13w16['znl']

array([ 0.35420001,  0.42559999,  0.38780001,  0.13850001,  0.51849997,
        0.3488    ,  0.6153    ,  0.53460002,  0.68279999,  0.28889999,
        0.2823    ,  0.1672    ,  0.68650001,  0.2694    ,  0.46849999,
        0.55739999,  0.2261    ,  0.373     ,  0.72920001,  0.7392    ,
        0.34509999,  0.27399999,  0.0985    ,  0.92699999,  1.03999996], dtype=float32)

### Build the systems

In [39]:
all_lls = []
for ii,qso in enumerate(l13w16['sightlinenl']):
    # init
    lls = LLSSystem(radec=l13_radec[qso],zem=l13_zem[qso],
                    zabs=l13w16['znl'][ii],NHI=l13w16['nhinl'][ii],
                    sig_NHI=l13w16['enhinl'][ii], 
                    vlim=[-500,500]*u.km/u.s)
    # Append
    all_lls.append(lls)

### Finish

## Wotta+16

### Metallicity

In [7]:
std = readsav('/Users/xavier/Desktop/final_metallicities.save')

In [11]:
std.keys()

['sightlinenl',
 'ic95lownlrc',
 'ic68lowcw',
 'enhinlrc',
 'icmeanallrc',
 'ic95highcw',
 'w2796cw',
 'em_nmgiinl',
 'w2803cw',
 'ep_metcw',
 'ep_nmgiicw',
 'ep_nmgiiallrc',
 'nmgiicw',
 'em_nmgiinlrc',
 'ep_metnlrc',
 'zcw',
 'ep_nmgiiall',
 'ep_metnl',
 'sightlinenlrc',
 'zallrc',
 'ic95lowallrc',
 'em_nmgiicw',
 'enhiall',
 'ic68highnlrc',
 'nhinl',
 'metnl',
 'enhiallrc',
 'ew2803cw',
 'ic95lowcw',
 'enhinl',
 'zall',
 'em_nmgiiallrc',
 'icmeannlrc',
 'em_metcw',
 'icmeancw',
 'nhiall',
 'nhinlrc',
 'icdiffnlrc',
 'iclogu3nlrc',
 'ic95highnlrc',
 'nhiallrc',
 'metnlrc',
 'ep_nmgiinlrc',
 'em_metnlrc',
 'ep_metall',
 'znlrc',
 'ew2796cw',
 'em_nmgiiall',
 'em_metall',
 'metallrc',
 'metcw',
 'icdiffallrc',
 'ic68highcw',
 'em_metnl',
 'iclogu3allrc',
 'nhicw',
 'sightlineall',
 'ic68highallrc',
 'enhicw',
 'sightlineallrc',
 'detectorcw',
 'nmgiiall',
 'iclogu3cw',
 'nmgiiallrc',
 'metall',
 'ic95highallrc',
 'ic68lownlrc',
 'znl',
 'ic68lowallrc',
 'ep_metallrc',
 'em_metallrc',
 

In [ ]:
std['znlrc']